# Building an MDMC Universe
An MDMC simulation requires a configuration and a topology defined within a `Universe` object. Although MDMC is somewhat flexible with regards to the order in which these are created, a suggested approach is as follows:

*Restriction on creating a* `Universe` *object*: MDMC currently only supports orthorhombic boxed Universes; the `dimensions` must be specified when creating (initialising) the `Universe` object:

In [1]:
# Import the Universe class
from MDMC.MD import Universe
# Initialise a Universe with dimensions in Ang
universe = Universe([10.0, 15.0, 20.0])

Universe created with:
  Dimensions       [10.0, 15.0, 20.0]
  Force field                    None
  Number of atoms                   0



To create a cubic `Universe`, only a single float needs to be specified for the `dimensions`:

In [2]:
universe = Universe(10.0)

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0



## Create an atomic configuration
Configurations can either be specified by the user or read from a CIF file.

### Create an atom
Each atom is specified using an `Atom` object, which possesses (amongst other things) a `position`, `velocity`, `element`, `mass`, `charge`, and `atom_type`.  At a minimum the `element` must be specified when creating an `Atom` object:

In [3]:
# Import the Atom class
from MDMC.MD import Atom
# Create a hydrogen atom
H1 = Atom('H')

This will create a Hydrogen atom with `mass` determined from an elemental lookup table, `position` set to the origin, `velocity` set to (0., 0., 0.), no `charge` and no `atom_type`. Note that if set, the velocity of atoms in the MD engine will be scaled when creating a `Simulation` in order to ensure the temperature is accurate. Otherwise, if the velocities of all `Atom` objects in a `Simulation` are 0, then the velocities of the atoms in the MD engine will be randomly chosen in order to provide an accurate temperature. For more details see [Running a Simulation](running-a-simulation.ipynb).

Atoms can also be created by copying another atom and passing the position of the new atom:

In [4]:
H2 = H1.copy(position=[1., 1., 1.])

The copied atom will have identical properties to the original attribute, except with a different ID (which is unique for all structural units) and a different `position`.  This includes interactions, which will apply to the copied atom in the same way as the original atom e.g. if H1 is bonded to an atom O, H2 will also be bonded to atom O.  An example of this is shown in the section **'Create bonded interactions'**.

### Atom type and ID
Each atom has an `atom_type`, which is used for applying non-bonded interactions between atoms; all atoms of the same `atom_type` will have the same non-bonded interactions applying to them.  The `atom_type` can be specified when the atom is created:

In [5]:
O1 = Atom('O', position=[8.0, 8.0, 8.0], atom_type=1)
# and to add the created atom to universe do
universe.add_structural_unit(O1)

Alternatively, the `atom_type` can be inferred by MDMC.  This occurs when an atom with no defined `atom_type` is added to a Universe. Additionally, when adding a `StructuralUnit` to the `Universe`, its `ID` can be used instead of the object itself:

In [6]:
O2 = Atom('O', position=[9.0, 9.0, 9.0])
print('ID of `O2` is {}'.format(O2.ID))
universe.add_structural_unit(O2.ID)

ID of `O2` is 4


Assuming no other atoms have been added to the universe, this sets ```O2.atom_type``` equal to 1.  MDMC infers the `atom_type` of each atom based on its element and its interactions: all atoms with the same element and the same interactions **when they are added to the universe** will have the same `atom_type`.  **Once an atom's** `atom_type` **is set, it is immutable i.e. it cannot be changed.**
It is recommended that either all atom types are specified when atoms are created or none are (i.e. either MDMC is left to infer and assign all atom types or no atom types.  Only specifying some atom types could result in unexpected behaviour; if it is absolutely necessary then ensure that all atoms with specified atom types are added to the universe first).

To see what atoms have been added to the universe:

In [7]:
universe.structure_list

[<Atom
  {name: 'O',
   ID: 4,
   element: 'O',
   position: UnitNDArray([9., 9., 9.]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>,
 <Atom
  {name: 'O',
   ID: 3,
   element: 'O',
   position: UnitNDArray([8., 8., 8.]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>]

You can see other attributes of the universe (or any other MDMC containers) with:

In [8]:
help(universe)

Help on Universe in module MDMC.MD.simulation object:

class Universe(MDMC.MD.container.AtomContainer)
 |  Universe(dimensions, force_field=None, structures=None, **settings)
 |  
 |  Class where the configuration and topology are defined
 |  
 |  Parameters
 |  ----------
 |  dimensions : numpy.ndarray, list, float
 |      Dimensions of the ``Universe``, in units of ``Ang``. A `float` can be
 |      used for a cubic universe.
 |  force_field : ForceField, optional
 |      A force field to apply to the Universe. The force fields available are:
 |      SPC, TIP3PFB, SPCE, OPLSAA, TIP3P. Default is None.
 |  structures : list, optional
 |      Either ``StructuralUnit`` objects contained in the ``Universe`` or their
 |      ``ID`` (`int`). Default is None.
 |  **settings
 |      ``kspace_solver`` (`KSpaceSolver`)
 |          The k-space solver to be used for both electrostatic and dispersive
 |          interactions. If this is passed then no ``electrostatic_solver`` or
 |          ``disp

The same applies for methods, for example:

In [9]:
help(universe.add_structural_unit)

Help on method add_structural_unit in module MDMC.MD.simulation:

add_structural_unit(structural_unit, force_field=None, center=False) method of MDMC.MD.simulation.Universe instance
    Adds a single ``StructuralUnit`` to the ``Universe``, with optional
    ``ForceField`` applying only to that ``StructuralUnit``
    
    Parameters
    ----------
    structural_unit : StructuralUnit or int
        The ``StructuralUnit`` (or its ``ID`` as an `int`) to be added to
        the ``Universe``
    force_field : str, optional
        The force field to be applied to the structural_unit. The available
        ``ForceField`` are:
        SPC, TIP3PFB, SPCE, OPLSAA, TIP3P
    center : bool, optional
        Whether to center `structural_unit` within the Universe as it is
        added



### Create bonded interactions
There are three bonded interaction types within MDMC: `Bond`, `BondAngle` and `Dihedral`.  Each `Interaction` must have an `InteractionFunction` which describes the `Interaction`.

### `Bond`
Below this is demonstrated where a `Bond` with a `HarmonicPotential` function is created.

In [10]:
# Import Bond and HarmonicPotential
from MDMC.MD import Bond
from MDMC.MD import HarmonicPotential
# Create a Bond with a HarmonicPotential
# The first argument in the HarmonicPotential is the equilibrium state and the second is the potential strength
HH_bond = Bond(H1, H2, function=HarmonicPotential(1., 100., interaction_type='bond'))

To see which units are supported:

In [11]:
from MDMC.common.units import SYSTEM
SYSTEM

{'LENGTH': 'Ang',
 'TIME': 'fs',
 'MASS': 'amu',
 'CHARGE': 'e',
 'ANGLE': 'deg',
 'TEMPERATURE': 'K',
 'ENERGY': 'kJ / mol',
 'FORCE': 'kJ / Ang mol',
 'PRESSURE': 'Pa',
 'ENERGY_TRANSFER': 'meV',
 'ARBITRARY': 'arb'}

### `BondAngle`

A `BondAngle` is created in the same manner except it requires a minimum of three atoms, and as with adding a `StructuralUnit` to the `Universe` either the object itself or its `ID` can be passed. The central atom should be the 2nd atom. For example, a water molecule would have:

In [12]:
from MDMC.MD import BondAngle

H1 = Atom('H')
H2 = Atom('H', position=[0., 1.63298, 0.])
O = Atom('O', position=[0., 0.81649, 0.57736])
HOH_angle = BondAngle(H1, O, H2)

# The following is equivalent
HOH_angle = BondAngle(H2.ID, O.ID, H1.ID)

Currently `HarmonicPotential` is the only `InteractionFunction` that can be applied to either `Bond` or `BondAngle`.

All `Bond` and `BondAngle` can be constrained; this can either be set when creating the `Bond` (or `BondAngle`) or afterwards:

In [13]:
HH_bond.constrained = True
HOH_angle = BondAngle(H1, O, H2, constrained=True)

For a constraint to be applied during a simulation, the `Universe` must have a `ConstraintAlgorithm`.

### `DihedralAngle`

A `DihedralAngle` is also created in the same manner, except it requires four atoms.  A `DihedralAngle` can be proper or improper, as specified by `DihedralAngle.improper`.  By default a `DihedralAngle` is proper.

The atoms in a proper `DihedralAngle` must be specified so that the 2nd and 3rd atoms are the central two atoms:

In [14]:
from MDMC.MD import DihedralAngle

C1 = Atom('C', position=[5.12033922, 4.63847287, 4.94075943])
N = Atom('N', position=[5.12991894, 3.78609704, 3.79996577])
C2 = Atom('C', position=[4.91462725, 4.08992816, 2.5091264])
O = Atom('O', position=[4.67405373, 5.24130678, 2.1180462])
proper = DihedralAngle(atoms=[C1, N, C2, O])

# For a proper DihedralAngle, the equivalent atom order is:
proper = DihedralAngle(atoms=[O, C2, N, C1])

The atoms in an improper `DihedralAngle` must be specific so that the 1st atom is the central atom.

In [15]:
N = Atom('N', position=[4.97080909, 2.91075722, 1.57280005])
C1 = Atom('C', position=[5.12033922, 4.63847287, 4.94075943])
C2 = Atom('C', position=[5.12991894, 3.78609704, 3.79996577])
C3 = Atom('C', position=[4.91462725, 4.08992816, 2.5091264 ])
improper = DihedralAngle(atoms=[N, C1, C2, C3], improper=True)

# The following are some of the equivalent permutations
# The only unique atom location is the first location (central atom)
improper = DihedralAngle(atoms=[N, C2, C1, C3], improper=True)
improper = DihedralAngle(atoms=[N, C3, C1, C2], improper=True)
improper = DihedralAngle(atoms=[N, C1, C3, C2], improper=True)

Currently `Periodic` is the only `InteractionFunction` that can be applied to `DihedralAngle` interactions.

### Copying bonded atoms

As mentioned above in the **'Create an atom'** section, if an `Atom` with a `BondedInteraction` is copied, the new atom will also have the same `BondedInteraction` (and be bonded to the same atom or atoms as the original).  For example:

In [16]:
wH1 = Atom('H', position=(5., 5., 5.))
wO = Atom('O', position=(5., 6.63298, 5.))
wBond = Bond((wH1, wO), function=HarmonicPotential(1., 100., interaction_type='bond'))
wH2 = wH1.copy(position=(0., 0.81649, 0.57736))

Both atoms `wH1` and `wH2` are bonded to `wO`:

In [17]:
wBond.atoms

[(<Atom
   {name: 'H',
    ID: 16,
    element: 'H',
    position: UnitNDArray([5., 5., 5.]) Ang,
    velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>,
  <Atom
   {name: 'O',
    ID: 17,
    element: 'O',
    position: UnitNDArray([5.     , 6.63298, 5.     ]) Ang,
    velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>),
 (<Atom
   {name: 'H',
    ID: 18,
    element: 'H',
    position: UnitNDArray([0.     , 0.81649, 0.57736]) Ang,
    velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>,
  <Atom
   {name: 'O',
    ID: 17,
    element: 'O',
    position: UnitNDArray([5.     , 6.63298, 5.     ]) Ang,
    velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>)]

### Create a molecule
A molecule consists of two or more atoms and at least one bonded interaction:

In [18]:
# Import the Atom, Molecule, Bond, BondAngle and HarmonicPotential classes
from MDMC.MD import Molecule, BondAngle
# Create a H2 molecule
H1.position = [0., 0., 0.]
H2.position = [1., 1., 1.]
H_mol = Molecule(position=[2., 1.5, 1.], atoms=[H1, H2], interactions=[HH_bond])

When a `Molecule` is created, the position of the atoms relative to one another is fixed.  The atoms are then moved so that the position of the molecular centre of mass is what was passed when creating the molecule.  In the example above, the atoms were at `[0., 0., 0.]` and `[1., 1., 1.]` before `H_mol` was created; therefore they will always be separated by `1.0 Ang` in each dimension, no matter where the molecule is moved to.  The molecular centre of mass is set to `[2., 1.5, 1.]`, so the atom positions are changed to `[2., 1.5, 1.]` and `[3., 2.5, 2.]` respectively.
It is also possible to `copy` molecules:

In [19]:
H_mol2 = H_mol.copy(position=[5., 5., 5.])

When a `Molecule` is copied, each `Atom` is copied, as are all of the bonded interactions between these atoms (and all of the non-bonded interactions).

One method for building molecules is to copy atoms, as the interactions are also copied. For example, to build methane:

In [20]:
m_C = Atom('C')
# Define the H atom positions relative to a C at [0,0,0]
d = 0.629118
H_pos = [[d, d, d], [-d, -d, d], [d, -d, -d], [-d, d, -d]]
m_H = Atom('H', position=H_pos[0])
# Add a bond between 
CH_bonds = Bond(m_C, m_H, function=HarmonicPotential(1.09, 100., interaction_type='bond'))
# Make three H atom copies (which are therefore all bonded to m_C)
H_atoms = [m_H]
for pos in H_pos[1:]:
    H_atoms.append(m_H.copy(pos))
# The next two lines simply create a list of unique HCH triplets
# e.g. [(m_H1, m_C, m_H2), (m_H1, m_C, m_H3), ..., (m_H3, m_C, m_H4)].
from itertools import combinations
HCH_triplets = [(i[0], m_C, i[1]) for i in combinations(H_atoms, 2)]
# Unpack the list of triplets with * notation i.e. [(...), (...), (...)] becomes (...), (...), (...)
HCH_angles = BondAngle(*HCH_triplets, function=HarmonicPotential(109.5, 10., interaction_type='angle'))
# Create a methane molecule by adding C atom to list of H atoms
methane = Molecule(atoms=[m_C]+H_atoms)

### Creating molecules using configuration files

In addition to creating molecules manually, it is also possible to create them from atomic configuration files (e.g CIF files).  Please see the tutorial [Reading atoms from configuration files](read-configurations.ipynb) for a detailed description on how to do this.

### Add structural units to a universe
There are two methods for adding a structural unit to a universe:

In [21]:
# Add an individual structural unit to the universe
universe.add_structural_unit(H_mol)
# Fill the universe with the structural unit repeated on a cubic lattice with a specific number density
universe.fill(H_mol, num_density = 0.01)

Currently the `fill` command cannot be used in conjunction with `add_structural_unit`. When using it with a cubic universe the density will be isotropic, however the exact number of structural units added by `fill` may be lower than expected as it will always add a cube number. Using `fill` with a non-cubic universe is not recommended as the density may or may not be isotropic depending on the dimensions of the universe and the number of units.

### Create non-bonded interactions
Non-bonded interactions are applied to atoms based on their `atom_type`, rather than to individual atoms.  They must also have a `Universe` specified, so that they know to which atoms they apply (atoms must have an `atom_type` once they have been added to a universe).  For example, to create a `Dispersion` interaction with a `LennardJones` interaction function between two atom types:

In [22]:
# Import the Dispersion interaction and Lennard-Jones function
from MDMC.MD import Dispersion
from MDMC.MD import LennardJones
# Create a dispersion interaction with a Lennard-Jones function between atoms with atom_type 1 (O) and atom_type 2 (H)
# The first LJ parameter is epsilon and the second is sigma
# The cutoff for the dispersion interaction can also be set (in Ang)
LJ_HO = Dispersion(universe, [1, 2], function=LennardJones(0.65, 3.), cutoff=10.)

The exception to this is `Coulombic` interactions, which can be applied either to a list of atoms or to a list of atom types.  If the Coulombic interaction is applied to a list of atoms, the universe does not need to be specified:

In [23]:
# Import the Coulombic interaction
from MDMC.MD import Coulombic
# Create a Coulombic interaction with a Coulomb potential and a charge of 0.42
c_H = Coulombic(atoms=[H1, H2], charge=0.42)

As `Coulombic` interactions typically have the same interaction function (i.e. a `Coulomb` function, where the force results in Coulomb's law), `Coulombic` interactions do not need to be specified with an interaction function (although other functions can be provided); to set the function as `Coulomb`, a value for the charge can be passed.  The warning highlights the `Coulomb` interaction function has been automatically set.  This is equivalent to:

In [24]:
from MDMC.MD import Coulomb
c_H = Coulombic(atoms=[H1, H2], function=Coulomb(0.42))

As with all non-bonded interactions, a Coulombic interaction can also be created by specifying the atom types:

In [25]:
c_O = Coulombic(universe, atom_types=[1], charge=0.42)

In this case a universe must be provided.

### Applying a `ForceField`

As well as manually setting the `Parameter` values for each interaction (e.g. `Coulombic(atoms=[H1, H2], function=Coulomb(0.42))` to set the `charge`), it is also possible to apply a `ForceField` to a `Universe`, in order to set the `Parameter` values.  Please see the tutorial [Applying a ForceField](applying-a-forcefield.ipynb) for a detailed description on how to do this.

### Adding kspace solvers to a universe
There are several solvers for determining the long range energy contribution for non-bonded interactions, including Ewald, particle-particle particle-mesh (PPPM), and particle-mesh Ewald (PME).  If the long range contribution to the non-bonded energy (i.e. > cutoff) is to be calculated during a simulation, a kspace solver has to be added to the universe.  A solver can be specified for either electrostatic or dispersive interactions, or both. This can either be during universe initialisation or afterwards:

In [26]:
# Import Ewald and PPPM kspace solvers
from MDMC.MD import Ewald, PPPM
# Create an ewald solver
ewald = Ewald(accuracy=1e-5)
# Initialise a universe with an Ewald solver for both electrostatics and dispersive interactions
uni1 = Universe(10., kspace_solver=ewald)
# Initialise a universe and then add a PPPM solver for electrostatic interactions
uni2 = Universe(10.)
pppm = PPPM(accuracy=1e-4)
uni2.electrostatic_solver = pppm
# Initialise a universe with a PPPM solver for dispersive interactions
uni3 = Universe(10., dispersive_solver=PPPM)

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0



Not all kspace solvers are implemented for all MD engines, and they may also require different parameters to be specified - see the MD engine documentation for more information.

### Adding constraint algorithms to a universe
In a similar manner to kspace solvers, constraint algorithms (e.g. SHAKE, RATTLE) can also be passed to a universe.  A constraint algorithm is required if any of the bonded interactions are constrained.

In [27]:
# Import Shake and Rattle
from MDMC.MD import Shake, Rattle
# Initialise a universe with the Shake algorithm
# The first Shake parameter is the accuracy and the second is the maximum number of iterations used for any constraint calculation
shake = Shake(1e-4, 100)
uni4 = Universe(10., constraint_algorithm=shake)
# Add Rattle after universe initialisation
rattle = Rattle(1e-5, 1000)
uni5 = Universe(10.)
uni5.constraint_algorithm = rattle

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0

Universe created with:
  Dimensions       [10.0, 10.0, 10.0]
  Force field                    None
  Number of atoms                   0



Not all constraint algorithms are implemented for all MD engines, and they may also required different parameters to be specified - see the MD engine documentation for more information.

## Example Universe filled with water
For water, SPCE, SPC, TIP3P and TIP3PFB forcefields are predefined, so parameters do not need to be set when interactions are defined; instead parameters are set by adding a forcefield to the universe.

Note that all water models have (harmonic) potential strengths defined for their `Bond`s and `BondAngle`s. In order to create a rigid water molecule in accordance with the models, a `constraint_algorithm` should be passed to the `Universe` and `constrained=True` passed to the relevant interactions, as shown below.

"Constraining" the ability of the bonds to oscillate during MD in this way should not be confused with constraining the value of an MDMC `Parameter` to a certain numerical range during refinement, as described in [Running a Refinement](running-a-refinement.ipynb). It is entirely possible to have a rigid bond but allow the length of that bond to change between refinement steps, or conversely have a bond that is free to oscillate during MD but the equilibrium length is not altered as part of the refinement.

In [28]:
from MDMC.MD import *

universe = Universe(dimensions=21.75, constraint_algorithm=Shake(1e-4, 100), electrostatic_solver=PPPM(accuracy=1e-5))
H1 = Atom('H')
H2 = Atom('H', position=(0., 1.63298, 0.))
O = Atom('O', position=(0., 0.81649, 0.57736))
H_coulombic = Coulombic(atoms=[H1, H2], cutoff=10.)
O_coulombic = Coulombic(atoms=O, cutoff=10.)
water_mol = Molecule(position=(0, 0, 0),
                     velocity=(0, 0, 0),
                     atoms=[H1, H2, O],
                     interactions=[Bond((H1, O), (H2, O), constrained=True),
                                   BondAngle(H1, O, H2, constrained=True)],
                     name='water')
universe.fill(water_mol, num_density=0.03356718472021752)
O_dispersion = Dispersion(universe, [O.atom_type, O.atom_type], cutoff=10., vdw_tail_correction=True)
universe.add_force_field('SPCE')

Universe created with:
  Dimensions       [21.75, 21.75, 21.75]
  Force field                       None
  Number of atoms                      0



## Accessing information
After creating a `Universe`, the various properties and attributes of it can be accessed as follows:

### Geometry

In [29]:
print('The dimension(s) of the Universe are {0}, giving a volume of {1}'
      ''.format(universe.dimensions, universe.volume))

The dimension(s) of the Universe are [21.75 21.75 21.75] Ang, giving a volume of 10289.109375 Ang ^ 3


### Forcefield

In [30]:
print('The current force field applied to the Universe is:\n{}'
      ''.format(universe.force_fields))

The current force field applied to the Universe is:
<SPCE
 {interaction_dictionary: {(<class 'MDMC.MD.interactions.Coulombic'>, ('O',)): <Coulomb
 {parameters: array([<Parameter
        {name: 'charge',
         value: -0.8476 e,
         unit: 'e',
         fixed: False,
         constraints: None,
         interactions_name: None,
         functions_name: None,
         tied: False}>           ], dtype=object)}>, (<class 'MDMC.MD.interactions.Coulombic'>, ('H',)): <Coulomb
 {parameters: array([<Parameter
        {name: 'charge',
         value: 0.4238 e,
         unit: 'e',
         fixed: False,
         constraints: None,
         interactions_name: None,
         functions_name: None,
         tied: False}>           ], dtype=object)}>, (<class 'MDMC.MD.interactions.Dispersion'>, ('O', 'O')): <LennardJones
 {parameters: array([<Parameter
        {name: 'epsilon',
         value: 0.6502 kJ / mol,
         unit: 'kJ / mol',
         fixed: False,
         constraints: None,
        

### Configuration of atoms and molecules

In [31]:
print('There are {0} atoms in the Universe:\n{1}'
      ''.format(universe.n_atoms, universe.atoms))

There are 1029 atoms in the Universe:
[<Atom
  {name: 'H',
   ID: 50,
   element: 'H',
   position: UnitNDArray([0., 0., 0.]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>
 <Atom
  {name: 'H',
   ID: 51,
   element: 'H',
   position: UnitNDArray([0.     , 1.63298, 0.     ]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>
 <Atom
  {name: 'O',
   ID: 52,
   element: 'O',
   position: UnitNDArray([0.     , 0.81649, 0.57736]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>          ...
 <Atom
  {name: 'H',
   ID: 1418,
   element: 'H',
   position: UnitNDArray([18.64285714, 18.64285714, 18.64285714]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>
 <Atom
  {name: 'H',
   ID: 1419,
   element: 'H',
   position: UnitNDArray([18.64285714, 20.27583714, 18.64285714]) Ang,
   velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>
 <Atom
  {name: 'O',
   ID: 1420,
   element: 'O',
   position: UnitNDArray([18.64285714, 19.45934714, 19.22021714]) Ang,
   velocity: UnitNDArray

In [32]:
print('There are {0} molecules in the Universe:\n{1}'
      ''.format(universe.n_molecules, universe.molecule_list))

There are 343 molecules in the Universe:
[<Molecule
 {name: 'water',
  ID: 53,
  formula: 'H2O',
  position: UnitNDArray([0.        , 0.81649   , 0.51275077]) Ang,
  velocity: UnitNDArray([0, 0, 0]) Ang / fs,
  bounding_box: <BoundingBox
 {min: UnitNDArray([0., 0., 0.]) Ang,
  max: UnitNDArray([0.     , 1.63298, 0.57736]) Ang,
  volume: 0.0 Ang ^ 3}>,
  atoms: [<Atom
 {name: 'H',
  ID: 50,
  element: 'H',
  position: UnitNDArray([0., 0., 0.]) Ang,
  velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>, <Atom
 {name: 'H',
  ID: 51,
  element: 'H',
  position: UnitNDArray([0.     , 1.63298, 0.     ]) Ang,
  velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>, <Atom
 {name: 'O',
  ID: 52,
  element: 'O',
  position: UnitNDArray([0.     , 0.81649, 0.57736]) Ang,
  velocity: UnitNDArray([0., 0., 0.]) Ang / fs}>]}>, <Molecule
 {name: 'water',
  ID: 57,
  formula: 'H2O',
  position: UnitNDArray([0.        , 0.81649   , 3.61989363]) Ang,
  velocity: UnitNDArray([0, 0, 0]) Ang / fs,
  bounding_box: <Bound

### Configuration of interactions

In [33]:
print('There are {0} total interactions in the Universe:\n{1}'
      ''.format(universe.n_interactions, universe.interactions))

There are 1032 total interactions in the Universe:
[<BondAngle
 {function: <HarmonicPotential
 {parameters: array([<Parameter
        {name: 'equilibrium_state',
         value: 109.47 deg,
         unit: 'deg',
         fixed: False,
         constraints: None,
         interactions_name: 'BondAngle',
         functions_name: 'HarmonicPotential',
         tied: False}>                       ,
       <Parameter
        {name: 'potential_strength',
         value: 383.0 kJ / mol rad ^ 2,
         unit: 'kJ / mol rad ^ 2',
         fixed: False,
         constraints: None,
         interactions_name: 'BondAngle',
         functions_name: 'HarmonicPotential',
         tied: False}>                       ], dtype=object)}>,
  constrained: True}>, <Bond
 {function: <HarmonicPotential
 {parameters: array([<Parameter
        {name: 'equilibrium_state',
         value: 1.0 Ang,
         unit: 'Ang',
         fixed: False,
         constraints: None,
         interactions_name: 'Bond',
        

In [34]:
print('There are {0} bonded interactions in the Universe:\n{1}'
      ''.format(universe.n_bonded, universe.bonded_interactions))

There are 1029 bonded interactions in the Universe:
[<BondAngle
 {function: <HarmonicPotential
 {parameters: array([<Parameter
        {name: 'equilibrium_state',
         value: 109.47 deg,
         unit: 'deg',
         fixed: False,
         constraints: None,
         interactions_name: 'BondAngle',
         functions_name: 'HarmonicPotential',
         tied: False}>                       ,
       <Parameter
        {name: 'potential_strength',
         value: 383.0 kJ / mol rad ^ 2,
         unit: 'kJ / mol rad ^ 2',
         fixed: False,
         constraints: None,
         interactions_name: 'BondAngle',
         functions_name: 'HarmonicPotential',
         tied: False}>                       ], dtype=object)}>,
  constrained: True}>, <Bond
 {function: <HarmonicPotential
 {parameters: array([<Parameter
        {name: 'equilibrium_state',
         value: 1.0 Ang,
         unit: 'Ang',
         fixed: False,
         constraints: None,
         interactions_name: 'Bond',
       

In [35]:
print('There are {0} nonbonded interactions in the Universe:\n{1}'
      ''.format(universe.n_nonbonded, universe.nonbonded_interactions))

There are 3 nonbonded interactions in the Universe:
[<Coulombic
 {function: <Coulomb
 {parameters: array([<Parameter
        {name: 'charge',
         value: 0.4238 e,
         unit: 'e',
         fixed: False,
         constraints: None,
         interactions_name: 'Coulombic',
         functions_name: 'Coulomb',
         tied: False}>                  ], dtype=object)}>,
  atom_types: [1],
  cutoff: 10.0 Ang}>, <Coulombic
 {function: <Coulomb
 {parameters: array([<Parameter
        {name: 'charge',
         value: -0.8476 e,
         unit: 'e',
         fixed: False,
         constraints: None,
         interactions_name: 'Coulombic',
         functions_name: 'Coulomb',
         tied: False}>                  ], dtype=object)}>,
  atom_types: [2],
  cutoff: 10.0 Ang}>, <Dispersion
 {function: <LennardJones
 {parameters: array([<Parameter
        {name: 'epsilon',
         value: 0.6502 kJ / mol,
         unit: 'kJ / mol',
         fixed: False,
         constraints: None,
         in

### Constraints

In [36]:
print('The current constraint algorithm applied to the Universe is {}'
      ''.format(universe.constraint_algorithm.name))

The current constraint algorithm applied to the Universe is Shake


### Solvers
Note that a `kspace_solver` is mutually excusive with the other solver types.

In [37]:
print('The kspace_solver is {}'.format(universe.kspace_solver and universe.kspace_solver.name))
print('The electrostatic_solver is {}'.format(universe.electrostatic_solver and universe.electrostatic_solver.name))
print('The dispersive_solver is {}'.format(universe.dispersive_solver and universe.dispersive_solver.name))

The kspace_solver is None
The electrostatic_solver is PPPM
The dispersive_solver is None


### Parameters

In [38]:
print('The MDMC parameters of the Universe are:\n{}'
      ''.format(universe.parameters))

The MDMC parameters of the Universe are:
[<Parameter
 {name: 'epsilon',
  value: 0.6502 kJ / mol,
  unit: 'kJ / mol',
  fixed: False,
  constraints: None,
  interactions_name: 'Dispersion',
  functions_name: 'LennardJones',
  tied: False}>, <Parameter
 {name: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, <Parameter
 {name: 'sigma',
  value: 3.166 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Dispersion',
  functions_name: 'LennardJones',
  tied: False}>, <Parameter
 {name: 'potential_strength',
  value: 383.0 kJ / mol rad ^ 2,
  unit: 'kJ / mol rad ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>, <Parameter
 {name: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interac